In [ ]:
import shutil
import os
import numpy as np
from datetime import datetime
import matplotlib.pylab as plt
import tensorflow as tf

import sys
sys.path.append(os.path.abspath("../framework"))
sys.path.append(os.path.abspath("../concrete"))

In [ ]:
from AsmAction import AsmAction
from AsmSimulator import AsmSimulator
from ConcObservation import ConcObservation
from ConcAgentFactory import ConcAgentFactory
from ConcBuildOrder import ConcBuildOrder
from ConcEnvrionmentFactory import ConcEnvironmentFactory
from ConcRewardGiverFactory import ConcRewardGiverFactory
from ConcTrainerFactory import ConcTrainerFactory
from ConcValueFunctionApproximatorFactory import ConcValueFunctionApproximatorFactory
from framework import Trainer, ObservationSequence

## S100: Check the responce of Asm Simulator to the actions

### S110: steady state with various set values of DO

In [ ]:
def checkSteadyState(action):
    # ignore the fluctuation of inflow
    asmSimulator =  AsmSimulator(h = 1.0)
    asmSimulator.init()

    Y = []
    X = []

    for _ in range(14): # 1[step] = [1day]
        asmSimulator.update(action)
        observation = asmSimulator.observe()
        Y.append(observation.getValue())
        X.append(asmSimulator.x)

    Ynumpy = np.concatenate(Y, axis=0) # (*, nPv = 1)
    Xnumpy = np.stack(X, axis=0) # (*, nAsm)

    plt.figure()
    plt.plot(Ynumpy)
    plt.title('NH4')

    for k1, name in enumerate(asmSimulator.asmVarNames):
        print("{0:5s} {1:8.2f}".format(name, asmSimulator.x[k1]))

#### DO = 3.0

In [ ]:
action = AsmAction(np.ones((1,1)) * 10.0) # (1, nMv = 1)
Do = action.getActionOnEnvironment()
checkSteadyState(action)

#### DO = 0.8

In [ ]:
action = AsmAction(np.ones((1,1)) * -1.0) # (1, nMv = 1)
Do = action.getActionOnEnvironment()
checkSteadyState(action)

#### DO = 0.14

In [ ]:
action = AsmAction(np.ones((1,1)) * -3.0) # (1, nMv = 1)
Do = action.getActionOnEnvironment()
checkSteadyState(action)

### S120: steady state with cyclic pertubated inflow

In [ ]:
def checkSteadyStateWithPertubation(action):
    # ignore the fluctuation of inflow
    asmSimulator =  AsmSimulator(amplitudePeriodicDv=2.0)
    asmSimulator.init()

    Y = []
    X = []

    for _ in range(96*3): # 96[step] = [1day]
        asmSimulator.update(action)
        observation = asmSimulator.observe()
        Y.append(observation.S_NH4)
        X.append(asmSimulator.x)

    Ynumpy = np.stack(Y, axis=0) # (*, nPv = 1)
    Xnumpy = np.stack(X, axis=0) # (*, nAsm)

    plt.figure()
    plt.plot(Ynumpy)
    plt.title('NH4')

#### DO = 3.0

In [ ]:
action = AsmAction(np.ones((1,1)) * 10.0) # (1, nMv = 1)
Do = action.getActionOnEnvironment()
checkSteadyStateWithPertubation(action)

#### DO = 0.8

In [ ]:
action = AsmAction(np.ones((1,1)) * -1.0) # (1, nMv = 1)
Do = action.getActionOnEnvironment()
checkSteadyStateWithPertubation(action)

#### DO = 0.36

In [ ]:
action = AsmAction(np.ones((1,1)) * -2.0) # (1, nMv = 1)
Do = action.getActionOnEnvironment()
checkSteadyStateWithPertubation(action)

### S130: closed loop simulation

In [ ]:
def runClosedLoopSimulation(nSteps):
    # ignore the fluctuation of inflow
    asmSimulator =  AsmSimulator(amplitudePeriodicDv=2.0, pgain=100, timeIntegral=1/24/4)
    asmSimulator.init()

    NH4 = []
    Do = []
    O2 = []

    gain = 2
    bias = -3.2
    
    for _ in range(nSteps): # 96[step] = [1day]
        
        observation = asmSimulator.observe()        
        u = observation.getValue() * gain + bias
        
        action = AsmAction(u)
        
        asmSimulator.update(action)
        observation = asmSimulator.observe()
        NH4.append(observation.S_NH4)
        Do.append(float(action.getActionOnEnvironment()))
        O2.append(asmSimulator.x[asmSimulator.idxSO2])

    NH4 = np.stack(NH4, axis=0) # (*, )
    Do = np.stack(Do, axis=0) #  (*, )
    O2 = np.stack(O2, axis=0) #  (*, )
    
    return NH4, Do, O2

In [ ]:
NH4, Do, O2 = runClosedLoopSimulation(96*3)

In [ ]:
#
plt.subplot(2,1,1)
plt.plot(Do, label = 'DO')
plt.plot(O2, label = 'S_O2')
plt.legend()
#
plt.subplot(2,1,2)
plt.plot(NH4)

## S200. Check the history of policy update

In [ ]:
def retrieveYURfromTrainer(trainer):

    Y = []
    for observationSequence in trainer.historyObservationSequences:
        y = observationSequence[-1].getValue() # (1, nPv)
        Y.append(y)
    Y = np.concatenate(Y, axis=0) # (*, nPv)

    U = []
    for action in trainer.historyActions:
        u = action.getActionOnEnvironment() # (1, nMv)
        U.append(u)
    U = np.concatenate(U, axis=0) # (*, nMv)

    R = []
    for reward in trainer.historyRewards:
        r = reward.getValue() # (1,)
        R.append(r)
    R = np.concatenate(R, axis=0) # (*,)
    
    return Y, U, R

In [ ]:
def exportTraceOfTraining(trainer, Gain, Bias):
    Y, U, R = retrieveYURfromTrainer(trainer)
    
    GainNumpy = np.concatenate(Gain, axis=0) # (*, nMv)
    BiasNumpy = np.stack(Bias, axis=0) # (*, nMv)

    fig = plt.figure(figsize=[25/2.57, 12/2.57])
    #
    plt.subplot(3,1,1)
    plt.plot(Y)
    plt.ylabel('NH4')
    #
    plt.subplot(3,1,2)
    plt.plot(U)
    plt.ylabel('DO')
    #
    plt.subplot(3,1,3)
    plt.plot(R)
    plt.ylabel('Reward')
    #
    plt.tight_layout()
    plt.savefig('trace.png')
    plt.close(fig)
    
    plt.figure(figsize=[25/2.57, 12/2.57])    
    plt.subplot(2,1,1)
    plt.plot(GainNumpy)
    plt.ylabel('Gain')
    plt.subplot(2,1,2)
    plt.plot(BiasNumpy)
    plt.ylabel('Bias')    
    plt.tight_layout()
    plt.savefig('parameter trace.png')
    plt.close(fig)

### SSS210: train an agent

In [ ]:
nIntervalPolicyOptimization = 2**1
buildOrder = ConcBuildOrder(nIteration=2**10
                            , nSeq=1
                            , nHorizonValueOptimization=nIntervalPolicyOptimization
                            , nIntervalPolicyOptimization=nIntervalPolicyOptimization
                            , nBatchPolicyOptimization=2**5
                            , nSaveInterval=2**5
                            , description="test"
#                             , tConstant = 10
                            , nHiddenValueApproximator = 2**3
                            , sdPolicy = 0.1
                            , nActionsSampledFromPolicy = 2**0                            
#                             , amplitudeDv = 0.0
                            , amplitudePeriodicDv = 2.0
                            , agentUseBias = True
                            , learningRateValueFunctionOptimizer = 1e-2
                            , weightOnError = 0.9
                            , policyOptimizer = "Adam"
                            , valueFunctionOptimizer = "Adam"
                            , returnType = "SumOfInfiniteRewardSeries"
                            , gamma = 0.9
                            , environmentName = "AsmSimulator"
                            )

agent = ConcAgentFactory().create(buildOrder)
environment = ConcEnvironmentFactory().create(buildOrder)
valueFunctionApproximator = ConcValueFunctionApproximatorFactory().create(buildOrder)
rewardGiver = ConcRewardGiverFactory().create(buildOrder)

trainerFactory = ConcTrainerFactory()
trainer = trainerFactory.create(agent, environment, valueFunctionApproximator, rewardGiver, buildOrder)

In [ ]:
trainer.init()
trainer.train(1)

agent.gainP.weights[0].assign(np.zeros((1,environment.nMv)))
agent.gainP.weights[1].assign(np.zeros((environment.nMv,)))

Gain = []
Bias = []

In [ ]:
t_bgn = datetime.now()
for k1 in range(2**16):
    sys.stdout.write('\r%s  %04d' % (datetime.now() - t_bgn, k1))
    gain = agent.gainP.weights[0].numpy() # (1, nMv)
    bias = agent.gainP.weights[1].numpy() # (nMv, )
    Gain.append(gain)
    Bias.append(bias)
    trainer.train(nIntervalPolicyOptimization)
    
    if k1 % 2**8 == 0:
        exportTraceOfTraining(trainer, Gain, Bias)